### openaiembeddings
* 문서의 내용을 수치적인 백터로 변환하는과정
* 문서의 의미를 수치화, 모델들이 문맥적 정보를 이해할수있게한다
* 벡터화된 문서들은 문서분류, 감정분석, 문서간 유사도 계산에 사용될수있다

지원되는 모델 목록

| MODEL                  | PAGES PER DOLLAR | PERFORMANCE ON MTEB EVAL | MAX INPUT |
|------------------------|------------------|---------------------------|-----------|
| text-embedding-3-small | 62,500           | 62.3%                     | 8191      |
| text-embedding-3-large | 9,615            | 64.6%                     | 8191      |
| text-embedding-ada-002 | 12,500           | 61.0%                     | 8191      |

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
text = "임베딩 테스트 하기위한 샘플 문장"

**쿼리 임베딩**
embeddings.embed_query(text)
* 주어진 텍스트를 임베딩벡터로 변환하는 함수
* 텍스트를 벡터공간에 매핑해, 의미적으로 유사한 텍스트를찾거나 유사도를 계산할때 사용

In [4]:
query_result = embeddings.embed_query(text)


In [7]:
query_result[:5]

[0.001385761657729745,
 0.03298284858465195,
 0.03938661888241768,
 -0.02090979553759098,
 0.01594056375324726]

###Document 임베딩**
embeddings.embed_documents()
* 함수를 사용하여 텍스트 문서를 임베딩
* [text]를 인자로 전달해 단일 문서를 리스트 형태로 임베딩 함수에 전달


In [8]:
doc_result = embeddings.embed_documents(
    [text, text, text, text]
)

In [9]:
len(doc_result)

4

In [11]:
doc_result[0][:5]

[0.001385761657729745,
 0.03298284858465195,
 0.03938661888241768,
 -0.02090979553759098,
 0.01594056375324726]

### 차원지정

* text-embedding-3-small모델같은경우 1536차원의 임베딩을 반환

In [12]:
len(doc_result[0])

1536

In [ ]:
# 차원이 1024인 text-embedding-3-smalll모델 생성
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions= 1024)

In [15]:
doc_result_1024 = embeddings_1024.embed_documents([text,text,text,text])
len(doc_result_1024[0])

1024

### 유사도 계산

In [ ]:
sentence1 = "안녕하세요 반갑습니다"
sentence2 = "안녕"
sentence3 = "안녕하세요 반가워요"
sentence4 = "hi, nice to meet u"
sentence5 = "i like to eat pizza"

In [22]:
pip install -q scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # 예시 문장들
# sentence1 = "이것은 첫 번째 문장입니다."
# sentence2 = "이것은 두 번째 문장입니다."
# sentence3 = "이 문장은 세 번째입니다."
# sentence4 = "여기는 네 번째 문장이 있습니다."
# sentence5 = "마지막 문장입니다."

# # 문장 리스트
# sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]

# # 문장 임베딩 (embeddings_1024가 정의되어 있다고 가정)
# embedded_sentences = embeddings_1024.embed_documents(sentences)

# # 코사인 유사도 계산
# similarity_matrix = cosine_similarity(embedded_sentences)

# # 유사도 행렬 출력
# print(similarity_matrix)

코사인유사도<br>
-1 ~1 사이의 값을 반환<br>


1에 가까울수록 더 유사<br>
0은 관계없음<br>
-1은 반대됨<br>

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentence = embeddings_1024.embed_documents(sentences)


In [25]:
def similarity(a,b):
    return cosine_similarity([a],[b])[0][0]

In [28]:
for i, sentence in enumerate(embedded_sentence):
    for j, other_sentence in enumerate(embedded_sentence):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.6356] 안녕하세요 반갑습니다 	 <=====> 	 안녕
[유사도 0.7309] 안녕하세요 반갑습니다 	 <=====> 	 안녕하세요 반가워요
[유사도 0.3497] 안녕하세요 반갑습니다 	 <=====> 	 hi, nice to meet u
[유사도 0.0747] 안녕하세요 반갑습니다 	 <=====> 	 i like to eat pizza
[유사도 0.6485] 안녕 	 <=====> 	 안녕하세요 반가워요
[유사도 0.4593] 안녕 	 <=====> 	 hi, nice to meet u
[유사도 0.1594] 안녕 	 <=====> 	 i like to eat pizza
[유사도 0.3656] 안녕하세요 반가워요 	 <=====> 	 hi, nice to meet u
[유사도 0.0943] 안녕하세요 반가워요 	 <=====> 	 i like to eat pizza
[유사도 0.2398] hi, nice to meet u 	 <=====> 	 i like to eat pizza
